# Chat Completions API

Chat Completions API는 OpenAI에서 제공하는 대화형 인공지능 모델(GPT 계열)을 활용해, 사용자의 메시지에 대해 자연스러운 대화 응답을 생성하는 API이다. 이 API는 챗봇, AI 비서, 자동화된 상담 시스템 등 다양한 대화형 서비스에 적용할 수 있다.


- **대화 문맥 유지**  
  Chat Completions API는 여러 메시지(대화 내역)를 입력받아, 이전 대화의 맥락을 이해하고 그에 맞는 응답을 생성한다. 즉, 단순히 한 문장만을 이어 쓰는 것이 아니라, 대화의 흐름을 반영하여 자연스러운 대화를 이어갈 수 있다.

- **역할(Role) 기반 메시지 구조**  
  입력 메시지는 배열 형태로 전달하며, 각 메시지는 `role`과 `content`로 구성된다.  
  - `system`: AI의 태도, 성격, 역할을 정의(예: "너는 친절한 도우미야.")
  - `user`: 사용자의 질문이나 요청
  - `assistant`: AI의 응답(이전 대화 내용 포함 가능)
  
  이 구조를 통해 AI의 응답 스타일이나 맥락을 세밀하게 제어할 수 있다[1][3][5].

**주요 파라미터 설명**

| 파라미터        | 설명                                                                 |
|----------------|----------------------------------------------------------------------|
| model          | 사용할 언어 모델명 (예: gpt-3.5-turbo, gpt-4o 등)                    |
| messages       | 대화 내역(역할/내용 포함) 배열                                        |
| max_tokens     | 생성할 응답의 최대 토큰 수(선택)                                     |
| temperature    | 창의성 조절(0~2, 낮을수록 일관성↑, 높을수록 다양성↑, 선택)           |
| top_p          | 누적 확률 기반 샘플링(temperature와 유사, 선택)                      |
| n              | 한 번에 생성할 응답 개수(선택)                                       |
| stop           | 응답 생성을 중단할 문자열 목록(선택)                                 |
| presence_penalty, frequency_penalty | 반복 억제 및 창의성 유도(선택)                 |
| user           | 사용자 식별자(선택, abuse monitoring 등 활용)                        |


- 위 예시에서 `messages` 배열에는 대화의 모든 메시지가 순서대로 들어가야 한다.  
- OpenAI는 이전 요청을 기억하지 않기 때문에, 매 API 호출마다 대화 내역 전체를 함께 보내야 한다.

In [2]:
from google.colab import userdata
import os
from openai import OpenAI

# OPENAI_API_KEY = userdata.get("OPENAI_API_KEY")
# client = OpenAI(api_key=OPENAI_API_KEY)

# 환경변수 지정
os.environ["OPENAI_API_KEY"] = userdata.get("OPENAI_API_KEY")
client = OpenAI()

## 대화형 챗봇

In [6]:
response = client.chat.completions.create(
    model='gpt-4o-mini',
    messages=[
        {'role': 'system', 'content': '너는 친절한 챗봇입니다.'},
        {'role': 'user', 'content': '안녕, 나는 차은우야~'},
        {'role': 'assistant', 'content': '안녕하세요, 차은우님! 만나서 반가워요. 어떻게 도와드릴까요?'},
        {'role': 'user', 'content': '잘 지냈어? 내 이름 기억하니?'}
    ]
)
print(response.choices[0].message.content)

네, 차은우님! 잘 지내셨나요? 어떻게 지내셨는지 궁금해요.


In [10]:
response = client.chat.completions.create(
    model='gpt-4o-mini',
    messages=[
        {'role': 'system', 'content': '너는 LLM 전문가입니다.'},
        {'role': 'user', 'content': '안녕, 나는 LLM 꿈나무 차은우야~'},
        {'role': 'assistant', 'content': '안녕하세요, 차은우님! LLM 꿈나무라니 멋지네요! 어떤 부분에 대해 이야기하고 싶으신가요? LLM에 대한 질문이나 궁금한 점이 있다면 언제든지 말씀해 주세요.'},
        {'role': 'user', 'content': 'Transformer모델을 공부하고 싶어.'},
        {'role': 'assistant', 'content': """좋아요! Transformer 모델은 자연어 처리(NLP) 분야에서 매우 중요한 혁신을 가져온 아키텍처입니다. 기본적인 구조와 작동 원리를 설명해드릴게요.

### 1. Transformer의 기본 구조
Transformer는 크게 두 가지 부분으로 구성되어 있습니다: **인코더(Encoder)**와 **디코더(Decoder)**.

- **인코더**: 입력 문장을 처리하고, 문장의 의미를 포착하는 고차원 표현을 생성합니다.
- **디코더**: 인코더의 출력을 바탕으로 출력 문장을 생성합니다. 주로 번역, 텍스트 요약 등에 사용됩니다.

### 2. 자기 주의 메커니즘 (Self-Attention)
Transformer의 핵심 개념 중 하나는 자기 주의 메커니즘입니다. 이는 입력 시퀀스 내의 단어들이 서로에게 얼마나 주의를 기울이는지를 학습하는 방법입니다. 이를 통해 문맥을 더 잘 이해할 수 있습니다.

### 3. 포지셔널 인코딩 (Positional Encoding)
Transformers는 순차적인 정보를 처리하지 않기 때문에 단어의 순서를 이해하기 위한 포지셔널 인코딩을 사용합니다. 이를 통해 단어의 위치 정보를 추가합니다.

### 4. 다중 헤드 주의 (Multi-head Attention)
Transformer는 하나의 자기 주의 메커니즘 대신 여러 개의 주의 헤드를 사용하여 서로 다른 표현을 학습합니다. 각 헤드는 다른 부분에 초점을 맞출 수 있어 정보의 다양성을 높입니다.

### 5. 피드포워드 신경망
각 인코더와 디코더의 구조에는 자기 주의 레이어 뒤에 위치한 피드포워드 신경망이 있습니다. 이는 각 단어의 표현을 독립적으로 처리하는 역할을 합니다.

### 6. Layer Normalization & Residual Connections
모든 레이어에는 레이어 정규화와 잔차 연결이 포함되어 있어, 훈련이 더 쉽게 이루어지도록 돕습니다.

### 학습 방법
Transformer 모델은 대량의 데이터를 통해 사전 학습(pre-training)되고, 특정 작업에 대해 미세 조정(fine-tuning)됩니다. 여러 가지 변종이 있으며, BERT, GPT 같은 모델들이 이 구조를 기반으로 하고 있습니다.

이해가 잘 되셨나요? 더 궁금한 부분이나 구체적인 질문이 있다면 말씀해 주세요!
"""},
        {'role': 'user', 'content': '어려워~ 어텐션을 초등학생도 이해할 수 있게 설명해줘.'},
        {'role': 'assistant', 'content': """물론이죠! 어텐션(attention)을 초등학생도 이해할 수 있게 쉽게 설명해볼게요.

### 어텐션을 설명하는 이야기

상상해보세요! 여러분이 친구들과 함께 학교에서 이야기를 나누고 있다고 합시다. 그 많은 친구들 중에서 누군가가 아주 중요한 이야기를 하고 있어요. 여러분은 그 친구의 말을 잘 듣고 싶어서 그 친구에게 집중합니다. 반면, 주변의 다른 소리나 대화는 잘 듣지 않으려고 해요. 이런 식으로 우리는 중요한 정보에 주의를 기울이고, 다른 것들은 잠깐 잊어버리는 거예요.

### 어텐션은 같아요!

어텐션은 컴퓨터가 텍스트를 읽을 때 어떻게 중요한 단어에 집중하는지를 설명하는 방법이에요. 예를 들어, "내가 어제 강아지와 함께 놀았어요"라는 문장이 있다고 해볼까요. 이 문장에서 "강아지"라는 단어는 이야기에 중요한 부분이에요. 어텐션 메커니즘은 이 "강아지"라는 단어에 더 많은 주의를 기울이고, 나머지 단어는 그만큼 덜 신경 쓰는 거죠.

### 간단한 예시

이제 조금 더 구체적인 예를 들어볼게요. 만약 여러분이 "사과는 빨갛고 맛있다"라는 문장을 읽고 있을 때, 어텐션은 "사과"라는 단어에 특히 주목하게 해요. 왜냐하면 이 단어가 문장에서 가장 중요한 의미를 갖고 있기 때문이죠.

어텐션은 이처럼 중요한 정보를 찾아내어 컴퓨터가 더 똑똑하게 텍스트를 이해하도록 도와주는 역할을 해요.

이해가 안 되거나 더 알고 싶은 부분이 있다면 언제든지 물어보세요!
"""},
        {'role':'user', 'content':'이해가 되는 것같아. 트랜스포머 내용을 요약해서 하나의 markdown문서로 정리해줘'}
    ]
)
print(response.choices[0].message.content)

좋아요! 어텐션(attention)을 초등학생도 이해할 수 있도록 쉽게 설명해볼게요.

### 어텐션을 이해하기 위한 비유: 친구의 이야기 듣기

상상해보세요. 친구가 재미있는 이야기를 할 때, 우리는 그 친구의 얼굴을 바라보며 집중하고 있다고 생각해요. 주변에서 다른 소리들이 나고, 여러 사람이 이야기할 수도 있지만, 우리는 친구가 말하는 내용을 잘 듣고 이해하기 위해 그 친구에게 더 많은 주의를 기울입니다.

#### 어텐션의 작동 원리

1. **모두가 주목받기를 원해요**:
   - 여러 친구들이 동시에 이야기하고 있어요. 그 중 한 친구가 "어제 놀이터에서 신기한 일이 있었어!"라고 할 때, 우리는 다른 소리보다 그 친구의 이야기에 더 집중합니다. 이때 그 친구가 중요한 이야기 주제인 거죠.

2. **중요한 부분에 더 집중해요**:
   - 그 이야기에서 "어제"와 "신기한 일" 같은 단어가 가장 중요할 수 있어요. 그래서 우리는 이 단어들에 더 많이 주목하고, 나머지 부분은 조금 덜 신경 쓰게 돼요. 친구가 이야기하는 동안, 우리는 중요한 정보에 더 집중하게 됩니다.

3. **다 같이 이야기할 수 있어요**:
   - 이렇게 다른 친구들의 이야기를 들으면서도 누군가의 이야기가 더 중요하다고 느끼면, 그 친구에게 더 많은 에너지를 쏟은 거예요. 그 친구는 우리가 미소를 지으면서 고개를 끄덕이는 것을 보고 자신감이 생기고, 더 이야기하고 싶어져요.

### Transformer의 어텐션

Transformer도 비슷한 방식으로 작동합니다. 문장을 구성하는 각 단어가 서로에게 얼마나 집중해야 하는지를 결정해요. 이 중심이 되는 단어를 선택하기 위해 어텐션 점수를 계산합니다. 그러면서 문장의 의미를 더 잘 이해하고, 필요한 정보에 주목하는 거죠.

### 요약

어텐션은 친구의 이야기를 들을 때처럼 중요한 정보에 더 집중하고, 나머지 부분은 조금 덜 신경 쓰는 방식이랍니다. 이런 식으로 Transformer는 문장을 이해하고 더 똑똑하게 대답할 수 있는

## 반복처리

In [13]:
# 대화내역을 로깅
messages = [
    {'role': 'system', 'content': '너는 친절한 챗봇입니다.'},
    {'role': 'user', 'content': '안녕, 나는 차은우야~'},
]

print('종료하려면, exit를 입력하세요...')
while True:
    # 사용자 입력
    user_input = input('User: ')

    if user_input.strip().lower() == 'exit':
        print('채팅을 종료합니다...')
        break

    # messages에 사용자 입력 추가
    messages.append({'role': 'user', 'content': user_input})
    # print(messages)

    # LLM 요청
    response = client.chat.completions.create(
        model = 'gpt-4o-mini',
        messages=messages,
        temperature=1
    )
    assistant_message = response.choices[0].message.content
    print(f'Assistant: {assistant_message}')

    # messages에 챗봇 출력 추가
    messages.append({'role': 'assistant', 'content': assistant_message})

종료하려면, exit를 입력하세요...
User: 안녕, 나 강윤구야
Assistant: 안녕하세요, 강윤구님! 만나서 반가워요. 오늘은 어떤 이야기를 나누고 싶으신가요?
User: 아이폰 시리즈 나열해줘
Assistant: 아이폰 시리즈는 여러 모델이 출시되었습니다. 아래는 주요 아이폰 모델의 리스트입니다:

1. **아이폰 (1세대)** - 2007년 출시
2. **아이폰 3G** - 2008년 출시
3. **아이폰 3GS** - 2009년 출시
4. **아이폰 4** - 2010년 출시
5. **아이폰 4S** - 2011년 출시
6. **아이폰 5** - 2012년 출시
7. **아이폰 5C** - 2013년 출시
8. **아이폰 5S** - 2013년 출시
9. **아이폰 6** - 2014년 출시
10. **아이폰 6 Plus** - 2014년 출시
11. **아이폰 6S** - 2015년 출시
12. **아이폰 6S Plus** - 2015년 출시
13. **아이폰 SE (1세대)** - 2016년 출시
14. **아이폰 7** - 2016년 출시
15. **아이폰 7 Plus** - 2016년 출시
16. **아이폰 8** - 2017년 출시
17. **아이폰 8 Plus** - 2017년 출시
18. **아이폰 X** - 2017년 출시
19. **아이폰 XS** - 2018년 출시
20. **아이폰 XS Max** - 2018년 출시
21. **아이폰 XR** - 2018년 출시
22. **아이폰 11** - 2019년 출시
23. **아이폰 11 Pro** - 2019년 출시
24. **아이폰 11 Pro Max** - 2019년 출시
25. **아이폰 SE (2세대)** - 2020년 출시
26. **아이폰 12** - 2020년 출시
27. **아이폰 12 Mini** - 2020년 출시
28. **아이폰 12 Pro** - 2020년 출시
29. **아이폰 12 Pro Max** - 2020년 출시
30. **아이폰 13** - 202

## stream

In [15]:
stream = client.chat.completions.create(
    model='gpt-4o-mini',
    messages=[{'role': 'user', 'content': '지금 stream테스트할거야, 아주 긴 응답 메세지를 보내줘.'}],
    stream=True
)
for chunk in stream:
    content = chunk.choices[0].delta.content
    if content is not None:
        print(content, end='')

물론입니다! 다음은 길고 다양한 이야기를 담고 있는 응답 메시지입니다.

---

옛날, 아주 먼 옛날에 작은 마을이 있었어요. 이 마을은 깊은 숲과 아름다운 호수에 둘러싸여 있었고, 마을 사람들은 평화롭게 살았어요. 그런데 어느 날, 마을 근처의 어두운 숲에서 이상한 소문이 퍼지기 시작했어요. 사람들이 그 숲을 지나가면 신비한 존재가 나타난다는 것이었죠.

호기심이 많은 소년, 민수는 친구들과 함께 숲의 구석구석을 탐험하기로 했어요. 그들은 모험을 통해 마을의 신화 속에 나오는 '숲의 정령'을 찾으려 했답니다. 민수는 희망에 가득 차 있었고, 친구들은 그의 아이디어에 흥미를 느끼며 동참했어요.

숲 속으로 들어가자, 그들은 아름다운 꽃들과 신비로운 나무들에 감탄했어요. 그러나 숲 깊숙한 곳으로 들어갈수록, 이상한 기분이 들기 시작했죠. 바람이 불고 나무들이 속삭이는 것처럼 느껴졌어요. 하지만 민수는 두려움을 이겨내고 계속해서 앞을 향해 나아갔어요.

그들이 숲의 중심에 도착했을 때, 갑자기 눈부신 빛이 그들을 감쌌어요. 그 빛 속에서 한 아름다운 정령이 나타났습니다. 그녀는 긴 머리와 반짝이는 날개를 가진 요정 같았어요. 민수와 친구들은 경악하며 발을 멈췄어요.

"여러분, 두려워하지 마세요," 정령이 부드러운 목소리로 말했어요. "저는 이 숲을 지키는 정령입니다. 여러분이 이곳까지 온 이유를 알고 있습니다."

민수는 떨리는 목소리로 대답했어요. "우리는 당신을 만나고 싶었어요! 숲의 신비를 알아보고 싶어서 왔어요!"

정령은 미소를 지으며 말했다. "이 숲은 많은 비밀과 이야기를 담고 있어요. 여러분이 진정으로 숲을 사랑하고 이해하려면, 세 가지 도전을 통과해야 해요."

민수와 친구들은 각자의 도전이 무엇인지 궁금해했어요. 정령은 이어서 세 가지 도전을 설명했어요.

첫 번째 도전은 '자연과의 연결'이었어요. 민수와 친구들은 각각의 나무와 꽃에 마음을 열고 그들과 대화해야 했어요. 두 번째 도전은 '친절의 시험'으로, 그들은 숲의 동물들에게 음식을 주고

In [16]:
# stream 버젼
messages = [
    {'role': 'system', 'content': '너는 친절한 챗봇이다.'}
]

print('종료하려면, exit를 입력하세요...')
while True:
    # 사용자 입력
    user_input = input('User: ')

    if user_input.strip().lower() == 'exit':
        print('채팅을 종료합니다...')
        break

    # messages에 사용자 입력 추가
    messages.append({'role': 'user', 'content': user_input})

    # LLM 요청
    stream = client.chat.completions.create(
        model='gpt-4.1',
        messages=messages,
        temperature=1,
        stream=True
    )
    print(f'Assistant: ', end='')
    for chunk in stream:
        content = chunk.choices[0].delta.content
        if content is not None:
            print(content, end='', flush=True) # 내부 buffer 사용하지 않음
    print()

    # messages 챗봇 출력 추가
    messages.append({'role': 'assistant','content': assistant_message})

종료하려면, exit를 입력하세요...
User: exit
채팅을 종료합니다...


# Token counting

비용 = ((입력 토큰수 * 단가) + (출력 토큰수 * 단가)) * 월 서비스 호출수

In [17]:
!pip install tiktoken

In [28]:
# 각 모델에 따른 토커나이져(인코딩) 가져오기
import tiktoken

gpt35 = tiktoken.encoding_for_model('gpt-3.5')
gpt4o = tiktoken.encoding_for_model('gpt-4o')
# gpt41 = tiktoken.encoding_for_model('gpt-4.1')
gpt41 = tiktoken.get_encoding('cl100k_base')

print(gpt35)
print(gpt4o)
print(gpt41)

<Encoding 'cl100k_base'>
<Encoding 'o200k_base'>
<Encoding 'cl100k_base'>


In [29]:
# 토큰수 세기
encoded_gpt35 = gpt35.encode('아버지가 방에 들어가신다.')
encoded_gpt4o = gpt4o.encode('아버지가 방에 들어가신다.')

print(len(encoded_gpt35))
print(len(encoded_gpt4o))

13
10


## 토큰비용 계산하기

In [30]:
text = """
더불어민주당이 6일 한동훈 국민의힘 대표가 윤석열 대통령 탄핵에 사실상 찬성 입장을 시사하자 7일로 예정됐던 윤석열 대통령의 탄핵소추안 표결을 앞당기는 방안을 고심하고 있다. 이재명 민주당 대표는 “한 대표의 입장을 정확하게 파악하는 것이 우선”이라며 신중한 태도를 보였다.

이 대표는 이날 기자들을 만나 윤 대통령 탄핵 표결을 앞당기는 방안에 대해 “지금 저렇게 불확실한 얘기를 믿고 미리 당겨서 협의를 할 필요가 있는가, 그런 생각이 일단 든다”고 말했다. 한 대표와의 회동 가능성에 대해서는 “요청은 했는데 아직 결정을 통보받지 못했다. (한 대표 측에서) 오후에 다시 연락하자는 연락이 왔다”고 전했다.

이 대표는 또 “사실 오늘 밤이 저는 매우 위험하다고 생각이 드는데, 제가 가진 감으로 본다면 오늘 밤 새벽에 또 뭔가 일을 벌이지 않을까 그런 걱정이 들긴 한다”며 ‘2차 계엄’ 가능성을 우려했다.

민주당은 이날 오전 한동훈 대표가 “윤 대통령의 조속한 직무 집행 정지가 필요하다”며 입장을 선회하자 긴급 의원총회를 열고 당내 의견 수렴에 나섰다. 의원총회에서는 탄핵소추안 표결 시점을 앞당기는 방안도 논의될 전망이다.

노종면 원내대변인은 비공개 최고위원 간담회가 끝난 뒤 “한 대표의 입장이 보도된 이후 긴장감이 높아지고 있고, 12월 3일 당일에 짐작했던 것 이상으로 치밀하게 의원, 정치인 체포 시도가 있었던 것과 이번 내란 사태에서 매우 중요한 작전이었던 걸로 파악되고 있다”며 “윤 대통령 옹위 세력이 어떻게 나올지 모르는 상황이라고 판단해 이런 비상한 상황 인식 떄문에 긴급 의원총회를 소집했다”고 전했다.

탄핵소추안 표결 시점 변경에 대해서는 “의장실에 본회의 일정 변경을 요청한 바는 아직 없다”며 “일단 신중하고 침착하게 대응할 것이고, 지금 한 대표 쪽의 입장이 뭔지 정확하게 파악하는 것이 우선이다. 필요하면 본회의를 앞당기는 방안도 의장실과 협조해서 추진할 수 있지만 아직은 결정된 바 없다”고 밝혔다.

민주당에서는 7일 오후 7시로 예정됐던 표결을 2시간 당겨 오후 5시에 추진하는 방안도 거론된다. 박성준 원내운영수석부대표는 이날 MBC 라디오 ‘김종배의 시선집중’ 인터뷰에서 “당초 오후 7시 정도 표결을 예상했는데 5시 정도는 해야 한다고 보고 있다”며 “국민의힘에서 탄핵소추안 투표 관련 상당한 지연 전략을 펼쳐서 시간을 늦출 수 있는 상황까지 고려하고 있다”고 설명했다.
"""

encoded_text_gpt4o = gpt4o.encode(text)
encoded_text_gpt41 = gpt41.encode(text)

print("gpt-4o 토큰수: ", len(encoded_text_gpt4o))
print("gpt-4.1 토큰수: ", len(encoded_text_gpt41))



gpt-4o 토큰수:  703
gpt-4.1 토큰수:  1215


In [31]:
response_gpt4o = client.chat.completions.create(
    model='gpt-4o',
    messages=[
        {'role': 'system', 'content': '너는 똑부러지는 시사/경제 전문가로써, 제공된 뉴스기사의 핵심을 잘 요약해서 정리해주는 챗봇이야.'},
        {'role': 'user', 'content': text}
    ],
    temperature=.2
)

response_gpt41 = client.chat.completions.create(
    model='gpt-4.1',
    messages=[
        {'role': 'system', 'content': '너는 똑부러지는 시사/경제 전문가로써, 제공된 뉴스기사의 핵심을 잘 요약해서 정리해주는 챗봇이야.'},
        {'role': 'user', 'content': text}
    ],
    temperature=.2
)

output_gpt4o = response_gpt4o.choices[0].message.content
output_gpt41 = response_gpt41.choices[0].message.content

print("gpt-4o 응답: ", output_gpt4o)
print("gpt-4.1 응답: ", output_gpt41)

print('\n', 'gpt-4o 토큰수: ', len(gpt4o.encode(output_gpt4o)))
print('gpt-4.1 토큰수: ', len(gpt41.encode(output_gpt41)))

gpt-4o 응답:  더불어민주당은 한동훈 국민의힘 대표가 윤석열 대통령 탄핵에 사실상 찬성하는 입장을 시사하자, 탄핵소추안 표결 시점을 앞당기는 방안을 검토 중입니다. 이재명 민주당 대표는 한 대표의 입장을 정확히 파악하는 것이 우선이라며 신중한 태도를 보이고 있습니다. 민주당은 한 대표의 발언 이후 긴급 의원총회를 열어 당내 의견을 수렴하고 있으며, 표결 시점을 당초 예정된 7일 오후 7시에서 5시로 앞당기는 방안도 논의되고 있습니다. 민주당은 국민의힘의 지연 전략 가능성을 염두에 두고 있으며, 상황에 따라 본회의 일정을 조정할 수 있다고 밝혔습니다.
gpt-4.1 응답:  ### 기사 핵심 요약

**1. 민주당, 윤석열 대통령 탄핵소추안 표결 시점 앞당기나**
- 더불어민주당이 7일로 예정된 윤석열 대통령 탄핵소추안 표결을 앞당기는 방안을 검토 중.
- 한동훈 국민의힘 대표가 윤 대통령 탄핵에 사실상 찬성하는 듯한 입장을 내비친 것이 계기.

**2. 이재명 대표, 신중론**
- 이재명 민주당 대표는 “한동훈 대표의 입장을 정확히 파악하는 것이 우선”이라며 신중한 태도.
- 한 대표와의 회동 요청은 했으나 아직 답변을 받지 못한 상태.

**3. 긴장 고조 및 ‘2차 계엄’ 우려**
- 이재명 대표는 “오늘 밤이 매우 위험하다”며, 추가적인 정치적 충돌(‘2차 계엄’) 가능성까지 언급.

**4. 민주당 긴급 의원총회 소집**
- 한동훈 대표의 입장 변화 이후 민주당은 긴급 의원총회를 열어 당내 의견을 수렴.
- 탄핵소추안 표결 시점을 앞당기는 방안도 논의 중.

**5. 표결 시점 변경 가능성**
- 현재로선 국회 본회의 일정 변경 요청은 없으나, 필요시 표결을 오후 5시로 앞당기는 방안이 거론됨.
- 국민의힘의 지연 전략에 대비해 표결 시간을 조정할 수 있다는 입장.

**핵심 정리**
- 민주당은 한동훈 대표의 발언을 계기로 윤 대통령 탄핵소추안 표결을 앞당길지 검토 중이나, 신중하게 상황을 파악하고 있음.
- 정치권 내 긴장감이 고조되고 있으며,

In [32]:
# 모델별 가격(2025년 6월 기준, 1M=1,000,000 토큰)
PRICING = {
    "gpt-4.1": {
        "input": 2.00,    # $2.00 / 1M input tokens
        "output": 8.00    # $8.00 / 1M output tokens
    },
    "gpt-4.1-mini": {
        "input": 0.40,    # $0.40 / 1M input tokens
        "output": 1.60    # $1.60 / 1M output tokens
    },
    "gpt-4.1-nano": {
        "input": 0.10,    # $0.10 / 1M input tokens
        "output": 0.40    # $0.40 / 1M output tokens
    },
    "o1": {
        "input": 2.00,    # $2.00 / 1M input tokens
        "output": 8.00    # $8.00 / 1M output tokens
    },
    "o3": {
        "input": 2.00,    # $2.00 / 1M input tokens
        "output": 8.00    # $8.00 / 1M output tokens
    },
    "o4-mini": {
        "input": 1.10,    # $1.10 / 1M input tokens
        "output": 4.40    # $4.40 / 1M output tokens
    },
    "gpt-4o": {
        "input": 2.50,    # $2.50 / 1M input tokens
        "output": 10.00   # $10.00 / 1M output tokens
    },
    "gpt-4o-mini": {
        "input": 0.15,    # $0.15 / 1M input tokens
        "output": 0.60    # $0.60 / 1M output tokens
    }
}

def count_tokens(text, model):
    encoding = tiktoken.encoding_for_model(model)
    encoded = encoding.encode(text)
    return len(encode)

def calc_cost(input_text, output_text, modol, num_service_call=1_000_000):
    input_tokens = count_tokens(input_text, model)
    output_tokens = count_tokens(output_text, model)


print(calc_cost(text, output_gpt4o, model='gpt-4o-mini'))
print(calc_cost(text, output_gpt41, model='gpt-4.1'))
